In [72]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import pandas as pd
from sklearn.utils import resample

In [40]:
fake1=pd.read_csv("/Volumes/Maxtor/fakeproteins/fakeres1.csv",delimiter=";",skiprows=1,encoding="iso-8859-1")
fake2=pd.read_csv("/Volumes/Maxtor/fakeproteins/fakeres2.csv",delimiter=";",skiprows=1,encoding="iso-8859-1")
fake3=pd.read_csv("/Volumes/Maxtor/fakeproteins/fakeres3.csv",delimiter=";",skiprows=1,encoding="iso-8859-1")
fake4=pd.read_csv("/Volumes/Maxtor/fakeproteins/fakeres4.csv",delimiter=";",skiprows=1,encoding="iso-8859-1")
fake5=pd.read_csv("/Volumes/Maxtor/fakeproteins/fakeres5.csv",delimiter=";",skiprows=1,encoding="iso-8859-1")
fake1["ID"]=fake1["ID"].str.replace("seq","").astype(int)
fake2["ID"]=fake2["ID"].str.replace("seq","").astype(int)+max(fake1["ID"])
fake3["ID"]=fake3["ID"].str.replace("seq","").astype(int)+max(fake2["ID"])
fake4["ID"]=fake4["ID"].str.replace("seq","").astype(int)+max(fake3["ID"])
fake5["ID"]=fake5["ID"].str.replace("seq","").astype(int)+max(fake4["ID"])
fake=pd.concat([fake1,fake2,fake3,fake4,fake5],ignore_index=True)

In [75]:
protein=pd.DataFrame(columns=['peptides','NB'])
c=0
for n in range(1,max(fake["ID"])):
    temp=fake.loc[fake["ID"]==n]
    if sum(temp["N_binders"])>0:
        index=temp.loc[temp["Rank"]==min(temp["Rank"])].index[0]
        if temp.loc[index,'Core']=='#Â¿NOMBRE?':
            pass
        else:
            protein.loc[c]=[temp.loc[index,'Core'],temp.loc[index,'N_binders']]
            c=c+1
    elif temp["Rank"].empty==True:
        pass
    else:
        index=temp.loc[temp["Rank"]==max(temp["Rank"])].index[0]
        if temp.loc[index,'Core']=='#Â¿NOMBRE?':
            pass
        elif temp.loc[index,"Rank"]<99:
            pass
        else:
            protein.loc[c]=[temp.loc[index,'Core'],temp.loc[index,'N_binders']]
            c=c+1
protein=protein.drop_duplicates(subset='peptides')
#protein.to_csv("/Volumes/Maxtor/fakeproteins/fakeresmerge.csv",index=False)

In [88]:
#Data Balance
data=protein
df_majority = data[data['NB']==0]
df_minority = data[data['NB']==1]
df_majority_downsampled = resample(df_majority, 
                                 replace=False,     # sample with replacement
                                 n_samples=df_minority['peptides'].size,    # to match majority class
                                 random_state=123) # reproducible results
data = pd.concat([df_minority, df_majority_downsampled])
data.to_csv("/Volumes/Maxtor/fakeproteins/fakeresmerge2.csv",index=False)